In [1]:
import numpy as np
import pandas as pd
import math

## The following method, calculates the price of an option with an underlying stock given:

## Binomial Tree Model for Option Pricing

The Binomial Tree model is a popular method used in financial mathematics to price options. This method involves dividing the time to expiry into discrete intervals or steps and simulating the possible paths the stock price could take over these intervals. The model can price various options, including European and American calls and puts, by considering the early exercise feature of American options.

In this implementation, the `Binomial_Trees` class encapsulates the logic for constructing a binomial tree, calculating stock prices at each node, and ultimately determining the option price based on the specified option type. The class is initialized with parameters defining the option and underlying stock characteristics, such as the stock's current price, volatility, time to expiry, risk-free interest rate, strike price, and option type.

### Variables Explained:
- `S0`: Initial stock price.
- `K`: Strike price of the option.
- `sigma`: Volatility of the stock, a measure of how much the stock price is expected to fluctuate.
- `T`: Time to expiry of the option, in years.
- `r`: Risk-free interest rate, annually compounded.
- `u`: Upward movement factor per time step.
- `d`: Downward movement factor per time step.
- `p`: Probability of an upward movement in the stock price.
- `spt`: A list to store stock prices at each node of the binomial tree.
- `opt`: A list to store option values at each node.
- `option_type`: Type of option ('ec' for European Call, 'ep' for European Put, 'ac' for American Call, 'ap' for American Put).

### Approach:
- **Stock Price Simulation**: The `stock_prices` method populates the `spt` list with simulated stock prices at each node, using the up (`u`) and down (`d`) factors derived from the volatility and delta time per step.
- **Option Value at Maturity**: The `option_value_mature` method calculates the option's value at maturity for the last nodes of the tree, based on the option type.
- **Backward Induction for Option Pricing**: The `option_prices` method and `option_value_at_timestept` function work together to calculate the option value at each node, using backward induction. This considers the possibility of early exercise for American options.


In [2]:
# Binomial Tree Model Implementation for Option Pricing
class Binomial_Trees():
    def __init__(self, stock_price, volatility, time_steps, time2expiry, interest_free_rate, strike_price, option_type='ec'):
        """Initialize the Binomial Tree model with given parameters."""
        # Option and stock parameters
        self.S0 = stock_price  # Initial stock price
        self.K = strike_price  # Strike price of the option
        self.sigma = volatility  # Stock volatility
        self.T = time2expiry  # Time to expiry in years
        self.r = interest_free_rate  # Risk-free interest rate
        self.option_type = option_type  # Type of option
        
        # Model parameters
        self.time_steps = min(time_steps, 10)  # Limit the number of time steps to at most 10
        self.delta_t = self.T / self.time_steps  # Time per step
        self.u = math.exp(self.sigma * math.sqrt(self.delta_t))  # Up factor
        self.d = math.exp(-self.sigma * math.sqrt(self.delta_t))  # Down factor
        self.p = (math.exp(self.r * self.delta_t) - self.d) / (self.u - self.d)  # Probability of up move
        
        # Lists to store stock prices and option values
        self.spt = []  # Stock prices at each node
        self.opt = []  # Option values at each node
        
        self.stock_prices()  # Generate stock prices
        self.option_prices()  # Calculate option prices

    def stock_prices(self):
        self.spt.append([self.S0])
        for ts in range(1,self.time_steps+1):
            self.spt.append(self.stock_price_at_timestept(ts))
            
    def stock_price_at_timestept(self, timestep):
        stock_prices = []
        for s in self.spt[timestep-1]:
            for move in range(2):
                if(move==0):
                    stock_prices.append(s*self.u)
                else:
                    stock_prices.append(s*self.d)
        return stock_prices

    def print_sp(self):
        num_values = 2**(self.time_steps-1)
        for spt in reversed(self.spt):
            for sp in spt:
                print(sp, end="   ")
            print()

    def print_op(self):
        num_values = 2**(self.time_steps-1)
        for opt in self.opt:
            for sp in opt:
                print(sp, end="   ")
            print()
            
    def option_value_mature(self):
        option_value = []
        assert self.option_type in ['ec','ep','ac','ap']
        if(self.option_type=='ec' or self.option_type=='ac'):
            for sp in self.spt[-1]:
                option_value.append(sp-self.K if sp-self.K>0 else 0.0)
        elif(self.option_type=='ep' or self.option_type=='ap'):
            for sp in self.spt[-1]:
                option_value.append(self.K-sp if sp-self.K<0 else 0.0)
        return option_value

    def option_prices(self):
        self.opt.append(self.option_value_mature())
        for ts in range(1,self.time_steps+1):
            self.opt.append(self.option_value_at_timestept(ts))

    def option_value_at_timestept(self, timestep):
        option_value = []
        s = self.opt[timestep-1]
        stock_prices_at_timestep = self.spt[self.time_steps - timestep]
        for idx in range(0,len(self.opt[timestep-1])//2):
            fu = s[2*idx]
            fd = s[2*idx + 1]
            f = math.exp(-self.r*self.delta_t)*(self.p*fu + (1-self.p)*fd)
            if self.option_type=='ec' or self.option_type=='ep':
                option_value.append(f)
            elif self.option_type=='ap':
                option_value.append(max(self.K - stock_prices_at_timestep[idx], f))
            elif self.option_type=='ac':
                option_value.append(max(stock_prices_at_timestep[idx] - self.K, f))
        return option_value

## Demonstration of Binomial Tree Model Across Option Types with Tailored Parameters

This section provides a practical demonstration of the `Binomial_Trees` class to estimate option prices for a spectrum of options: European Call (EC), European Put (EP), American Call (AC), and American Put (AP), each with its specific set of parameters. This nuanced approach allows for a more detailed examination of the model's flexibility in adapting to different market conditions and option strategies.

### Parameters for Demonstration:

For the sake of this demonstration, distinct parameters are utilized for call and put options, reflecting typical market scenarios that might favor one type of option over another based on volatility and strike price considerations.

- **Common Parameters**:
  - **Time Steps**: 3
  - **Time to Expiry**: 1 year
  - **Risk-Free Interest Rate**: 5%

- **Specific Parameters**:
  - **European Call (EC)**: Stock Price = `100`, Volatility = 20%, Strike Price = `100`
  - **European Put (EP)**: Stock Price = `100`, Volatility = 25%, Strike Price = `95`
  - **American Call (AC)**: Stock Price = `100`, Volatility = 20%, Strike Price = `105`
  - **American Put (AP)**: Stock Price = `100`, Volatility = 25%, Strike Price = `90`

### Demonstration Output:

The demonstration iterates over each option type, applying the specific parameters to illustrate how the `Binomial_Trees` model adapts the pricing strategy based on the option type and underlying market assumptions.

- For each option type, the output showcases:
  - The stock prices at each node of the binomial tree, demonstrating the potential paths the stock price could undertake.
  - The calculated option values at each node, using the binomial tree model. This includes considerations for early exercise in American options, showcasing the model's versatility in handling different option types and exercise strategies.

### Insights:

Through this demonstration, we gain valuable insights into the impact of varying market conditions (as represented by different volatilities and strike prices) on the pricing of options. The `Binomial_Trees` model provides a robust framework for analyzing these dynamics, offering critical insights for investors and traders alike.


In [3]:
# Example parameters common to all options
time_steps = 3
time2expiry = 1
interest_free_rate = 0.05

# Parameters specific to call and put options
params = {
    'ec': {'stock_price': 100, 'volatility': 0.2, 'strike_price': 100},
    'ep': {'stock_price': 100, 'volatility': 0.25, 'strike_price': 95},
    'ac': {'stock_price': 100, 'volatility': 0.2, 'strike_price': 105},
    'ap': {'stock_price': 100, 'volatility': 0.25, 'strike_price': 90}
}

option_types = ['ec', 'ep', 'ac', 'ap']

for opt_type in option_types:
    print(f"---\nDemonstrating {opt_type.upper()} Option:")
    p = params[opt_type]
    binomial_tree = Binomial_Trees(p['stock_price'], p['volatility'], time_steps, time2expiry, interest_free_rate, p['strike_price'], opt_type)
    print("Stock Prices at Each Node:")
    binomial_tree.print_sp()
    print("Option Values at Each Node:")
    binomial_tree.print_op()
    print("\n")


---
Demonstrating EC Option:
Stock Prices at Each Node:
141.39824580805166   112.24009024456677   112.24009024456677   89.09472522884109   112.24009024456676   89.09472522884107   89.09472522884109   70.72223522189249   
125.9783785810849   100.00000000000001   100.0   79.3787006360269   
112.24009024456676   89.09472522884107   
100   
Option Values at Each Node:
41.398245808051655   12.240090244566773   12.240090244566773   0.0   12.240090244566758   0.0   0.0   0.0   
27.631233198923145   6.545862681454766   6.545862681454758   0.0   
17.713888236329907   3.500653785088061   
11.043871091951106   


---
Demonstrating EP Option:
Stock Prices at Each Node:
154.18958055705855   115.52740254401431   115.52740254401432   86.55955020013666   115.52740254401431   86.55955020013664   86.55955020013664   64.855225391183   
133.46580738566723   100.00000000000001   100.0   74.92555730849976   
115.52740254401431   86.55955020013664   
100   
Option Values at Each Node:
0.0   0.0   0.0   8.440